# Treasure Hunt


Source: https://towardsdatascience.com/simple-reinforcement-learning-q-learning-fcddc4b6fe56

## Introduction
Reinforcement learning is a machine learning concept. Given an environment, the model attempts to understand the system with four basic principles. 
    1. States
    2. Actions
    3. Rewards
    4. Policy

An agent refers to who is learning the provided environment. This agent can encounter different types of states within this system which will help determine an action to select. Certain states provide various rewards which influence how our agent chooses some action. Lastly, the selection of actions is determined by some policy. 

In this notebook, we define a reinforcement learning model to select certain actions leveraging Q-learning as the policy.

## Defining the System
Treasure hunts consist of a brave hero, dangerous obstacles and glorious treasure respectively represented by our agent and various states. We also know our hero can move across the map to locate the treasure which represents our possible actions such as up, down, left and right. With our hero, we understand that dangerous obstacles represent a negative experience and treasures provide a positive experience. Thus the rewards are respectively negative and positive score values depending on what state our agent encounters. The following represents the list of defined system attributes.

    1. Model/Environment
       - A 2D matrix with indices representing locations with some state
    2. States
       - 0 represents a neutral block
       - 1 represents an extremely dangerous forest
       - 2 represents a monster
       - 3 represents a small coin
       - 4 represents the ultimate treasure room
    3. Actions
       - 0 is right
       - 1 is left
       - 2 is down
       - 3 is up
    4. Rewards
       - Neutral blocks do not receive any rewards
       - The forest blocks return -50 points
       - Monsters damage points by 4
       - Coins award 4 points
       - Treasure room presents 50 points
  
## Policy
For remembering and recording rewards given an action at some given state, Q-learning is leveraged. Essentially, the observed reward is saved based on the learning rate (e.g., how much the next step alters overall values) and gamma (e.g., opportunistic versus long-term reward) and the previously recorded reward. This is defined as follows:
* Q[curState, action] += learnRate * (reward + gamma * np.max(Q[newState]) - Q[curState, action])


In [1]:
#Imports
import pandas as pd
import numpy as np


In [2]:
#Defining the agent (Player) and system (Environment). These two classes work together.

class Player():
    def __init__(self, location=(0,0)):
        # initialize player's location
        self.Player=location
        
        # initialize score
        self.Score=0
    
    def setScore(self, reward=0):
        self.Score+=reward
        
    def setLocation(self, location):
        self.Player=location
        
    def updateLocation(self, action):
        # right=0, left=1, down=2, up=3
        if action[0]==0:
            self.Player=(self.Player[0],self.Player[1]+1)
        elif action[0]==1:
            self.Player=(self.Player[0],self.Player[1]-1)
        elif action[0]==2:
            self.Player=(self.Player[0]+1,self.Player[1])
        elif action[0]==3:
            self.Player=(self.Player[0]-1,self.Player[1])
        
    def getLocation(self):
        return self.Player
    
    def getScore(self):
        return self.Score
        
class Environment():    
    def __init__(self,size=5):
        # initialize matrix map
        self.mapEnv = np.zeros(shape=(size,size))
    
    def getSize(self):
        return self.mapEnv.shape[0]
    
    def setEnvironment(self):
        # set states in map... for now this is static, but could alter
        # [0 3 0 0 3]
        # [1 0 1 1 1]
        # [1 0 0 1 3]
        # [1 0 0 0 2]
        # [3 0 0 1 4]
        
        # 0 = blank space, 1 = wall/tree, 2 = monster, 3 = coins, 4 = treasure
        self.mapEnv[0]=[0, 3, 0, 0, 3]
        self.mapEnv[1]=[1, 0, 2, 1, 1]
        self.mapEnv[2]=[1, 0, 0, 1, 3]
        self.mapEnv[3]=[1, 0, 0, 0, 2]
        self.mapEnv[4]=[3, 0, 0, 1, 4]
        

    def getState(self,location):
        #remove tokens and monsters if encountered
        state=int(self.mapEnv[location[0]][location[1]])
        if state==2 or state==3: #monster or token
            self.mapEnv[location[0]][location[1]]=0
        return state

    def getReward(self,state): # define what each state represents for scoring
        if state == 0: #regular block
            return 0
        elif state == 1: #forest
            return -50
        elif state == 2: #monster
            return -4
        elif state == 3: #token
            return 4
        elif state == 4: #goal
            return 50

    def getPossibleActions(self,location):
        # returns list of possible states/actions player can move
    
        # create list of up, down, left, right
        # right=0, left=1, down=2, up=3
        possibleMoveSet=[(0,location[1]+1), (1,location[1]-1), (2,location[0]+1), (3,location[0]-1)]

        # filter list
        #*must be equal or greater than 0 for both cases (e.g., can't go off the map in the bottom left)
        #*must be equal or greater than the size for both cases (e.g., can't go off the map in the top right)
        possibleMoveSet=[move for move in possibleMoveSet if move[1] > -1 if move[1] < self.mapEnv.shape[0]]
        return possibleMoveSet
    
    def printOut(self,location, action=None):
        #now print out action taken
        # right=0, left=1, down=2, up=3
        if action==None:
            print("Start")
        elif action[0]==0:
            print("Action taken: Right")
        elif action[0]==1:
            print("Action taken: Left")
        elif action[0]==2:
            print("Action taken: Down")
        elif action[0]==3:
            print("Action taken: Up")
        
        # Map representation
        # [0 3 0 0 3]
        # [1 0 1 1 1]
        # [1 0 0 1 3]
        # [1 0 0 0 2]
        # [3 0 0 1 4]
        
        newMap=self.mapEnv.copy()
        newMap[location[0],location[1]] = 9
        print(newMap)
         

## Simulation
The end goal is the treasure room labelled as state 4. Thus, we must loop until the agent locates the reward. For each episode, an action is chosen and the map is printed with the new location of the agent (labelled 9). 

An important aspect to point out is exploration versus exploitation. To better understand certain actions at some reward, we must explore, but in order to get the best rewards, we typically want to exploit the actions that provide the highest possible reward. This is defined based on what we want and can be altered; however, for the intents and purposes of this course, I selected 0.6 such that upon random selection of a variable from range 0 to 1, will determine if a random action is chosen or based on our Q values.

In [3]:
def simulate(gameMap, player, Q):
    # Establish parameters for updating Q table and exploration/exploitation
    explorExploit=.6 #explore and exploit values
    learnRate=0.4 #based on https://en.wikipedia.org/wiki/Q-learning
    gamma=0.2 #determines focus on short term versus long term scores

    print("----- Iteration 0 -----")
    print("Player Score: "+str(player.getScore()))
    gameMap.printOut(player.getLocation()) #print out beginning

    curLoc= player.getLocation() # Get current location
    curState= gameMap.getState(curLoc) # Get current state
    iteration=0
    while curState!=4 and iteration!=-1: #loop until goal is reached or break if not located
        curActions= gameMap.getPossibleActions(curLoc) # Get possible actions
        eps=np.random.uniform(0, 1) # generate random value from 0 to 1
        if eps>=explorExploit: # grab random possible action for exploration
            index = np.random.randint(low=0,high=len(curActions), size=1)[0] #if returned list of multiple values, select the first
            action=curActions[index] # randomly grab an action
        else: # exploit highest value!
            #out of all possible actions, grab the one with the highest Q value
            chosen=[(-1,-1),float("-inf")] #format: ((action),Q-value)
            for act in curActions:
                if Q[curState,act[0]]>chosen[1]:
                    chosen=[act,Q[curState,act[0]]]
            action=chosen[0] #grab highest value

        # with the selected action, update the player's location
        player.updateLocation(action) # set the location to selected action
        newState=gameMap.getState(player.getLocation()) # grab the new state
        reward=gameMap.getReward(newState)
        player.setScore(reward) # set the score

        # Updating Q values
        Q[curState, action[0]] = Q[curState, action[0]] + learnRate * (reward + gamma * np.max(Q[newState]) - Q[curState, action[0]])

        #Increase iteration to prevent infinite loop
        iteration+=1

        #print out each change
        print("----- Iteration "+ str(iteration) + " -----")
        print("Player Score: "+str(player.getScore()))
        gameMap.printOut(player.getLocation(),action)

        #get current location and state
        curLoc= player.getLocation() # Get current location
        curState= gameMap.getState(curLoc) # Get current state


## Starting
The next few cells showcase how to setup the environment and player as well as implement the simulation with a few function calls. Q is established and passed prior as we want to check how the agent acts after learning the environment. 

In [4]:
# Setup environment and place player
gameMap = Environment()
gameMap.setEnvironment()
player = Player()
Q = np.zeros((5, 4)) # Initialize q-table values to 0. First four represent the four states and the second four represents the directions to move

simulate(gameMap, player, Q)
    

----- Iteration 0 -----
Player Score: 0
Start
[[9. 3. 0. 0. 3.]
 [1. 0. 2. 1. 1.]
 [1. 0. 0. 1. 3.]
 [1. 0. 0. 0. 2.]
 [3. 0. 0. 1. 4.]]
----- Iteration 1 -----
Player Score: 4
Action taken: Right
[[0. 9. 0. 0. 3.]
 [1. 0. 2. 1. 1.]
 [1. 0. 0. 1. 3.]
 [1. 0. 0. 0. 2.]
 [3. 0. 0. 1. 4.]]
----- Iteration 2 -----
Player Score: 4
Action taken: Right
[[0. 0. 9. 0. 3.]
 [1. 0. 2. 1. 1.]
 [1. 0. 0. 1. 3.]
 [1. 0. 0. 0. 2.]
 [3. 0. 0. 1. 4.]]
----- Iteration 3 -----
Player Score: 4
Action taken: Left
[[0. 9. 0. 0. 3.]
 [1. 0. 2. 1. 1.]
 [1. 0. 0. 1. 3.]
 [1. 0. 0. 0. 2.]
 [3. 0. 0. 1. 4.]]
----- Iteration 4 -----
Player Score: 4
Action taken: Right
[[0. 0. 9. 0. 3.]
 [1. 0. 2. 1. 1.]
 [1. 0. 0. 1. 3.]
 [1. 0. 0. 0. 2.]
 [3. 0. 0. 1. 4.]]
----- Iteration 5 -----
Player Score: 4
Action taken: Right
[[0. 0. 0. 9. 3.]
 [1. 0. 2. 1. 1.]
 [1. 0. 0. 1. 3.]
 [1. 0. 0. 0. 2.]
 [3. 0. 0. 1. 4.]]
----- Iteration 6 -----
Player Score: 8
Action taken: Right
[[0. 0. 0. 0. 9.]
 [1. 0. 2. 1. 1.]
 [1. 0. 0. 1.

In [5]:
print(Q)

[[ -0.09811639  -8.47625251   8.         -31.89617063]
 [-12.03671097 -20.           0.16827054   0.        ]
 [  0.           0.           0.           0.        ]
 [  0.           0.           0.           0.        ]
 [  0.           0.           0.           0.        ]]


In [6]:
#Reset player and use the new calculated Q values
gameMap2 = Environment()
gameMap2.setEnvironment()
player2 = Player()

simulate(gameMap2, player2, Q)


----- Iteration 0 -----
Player Score: 0
Start
[[9. 3. 0. 0. 3.]
 [1. 0. 2. 1. 1.]
 [1. 0. 0. 1. 3.]
 [1. 0. 0. 0. 2.]
 [3. 0. 0. 1. 4.]]
----- Iteration 1 -----
Player Score: -50
Action taken: Down
[[0. 3. 0. 0. 3.]
 [9. 0. 2. 1. 1.]
 [1. 0. 0. 1. 3.]
 [1. 0. 0. 0. 2.]
 [3. 0. 0. 1. 4.]]
----- Iteration 2 -----
Player Score: -100
Action taken: Down
[[0. 3. 0. 0. 3.]
 [1. 0. 2. 1. 1.]
 [9. 0. 0. 1. 3.]
 [1. 0. 0. 0. 2.]
 [3. 0. 0. 1. 4.]]
----- Iteration 3 -----
Player Score: -100
Action taken: Right
[[0. 3. 0. 0. 3.]
 [1. 0. 2. 1. 1.]
 [1. 9. 0. 1. 3.]
 [1. 0. 0. 0. 2.]
 [3. 0. 0. 1. 4.]]
----- Iteration 4 -----
Player Score: -100
Action taken: Right
[[0. 3. 0. 0. 3.]
 [1. 0. 2. 1. 1.]
 [1. 0. 9. 1. 3.]
 [1. 0. 0. 0. 2.]
 [3. 0. 0. 1. 4.]]
----- Iteration 5 -----
Player Score: -100
Action taken: Down
[[0. 3. 0. 0. 3.]
 [1. 0. 2. 1. 1.]
 [1. 0. 0. 1. 3.]
 [1. 0. 9. 0. 2.]
 [3. 0. 0. 1. 4.]]
----- Iteration 6 -----
Player Score: -100
Action taken: Right
[[0. 3. 0. 0. 3.]
 [1. 0. 2. 1. 1.